# Save `xarray.Datasets` as `pkl` 

In [11]:
import xarray as xr
import s3fs
import _pickle as pickle

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
from pathlib import Path 
env_path = Path('.') / '.env'

## Utils
**read_dataset**

In [3]:
def read_dataset(dataset_type, group, access_key_id, secret_accsess_key):
    # AWS S3 path
    s3_path = f's3://soils-revealed/{dataset_type}.zarr'
    # Initilize the S3 file system
    s3 = s3fs.S3FileSystem(key=access_key_id, secret=secret_accsess_key)
    store = s3fs.S3Map(root=s3_path, s3=s3, check=False)
    # Read Zarr file
    ds = xr.open_zarr(store=store, group=group, consolidated=True)
    
    # Change coordinates names
    ds = ds.rename({'x': 'lon', 'y': 'lat'})
    
    # Change depth coord from 0 to 1 dimensional array
    depths = ds.coords.get('depth').values
    if depths.ndim == 0: 
        ds = ds.squeeze().drop("depth")
        ds = ds.assign_coords({"depth": np.array([depths])})
        
    return ds 

**save_object**

In [4]:
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, -1)

## Save `xarray.Datasets`

In [12]:
#datasets = {'experimental-dataset': ['stocks', 'concentration'],
#           'global-dataset': ['historic', 'recent'],
#           }


datasets = {'crop_I': ['future'], 
            'crop_MG': ['future'], 
            'crop_MGI': ['future'], 
            'grass_part': ['future'], 
            'grass_full': ['future'], 
            'rewilding': ['future'], 
            'degradation_ForestToGrass': ['future'], 
            'degradation_ForestToCrop': ['future'], 
            'degradation_NoDeforestation': ['future']}


for dataset_type, groups in datasets.items():
    for group in groups:
        ds = read_dataset(dataset_type = dataset_type, group = group, 
                          access_key_id = os.getenv("S3_ACCESS_KEY_ID"), 
                          secret_accsess_key = os.getenv("S3_SECRET_ACCESS_KEY"))    
        
        save_object(ds, f'../data/{dataset_type}_{group}.pkl')

In [6]:
ds = read_dataset(dataset_type = 'global-dataset', group = 'recent', 
                  access_key_id = os.getenv("S3_ACCESS_KEY_ID"), 
                  secret_accsess_key = os.getenv("S3_SECRET_ACCESS_KEY"))  

In [7]:
ds

,Array,Chunk
Bytes,732.43 GB,4.72 MB
Shape,"(19, 60934, 158159)","(1, 477, 2472)"
Count,155649 Tasks,155648 Chunks
Type,float32,numpy.ndarray


In [8]:
with open(f'../data/{dataset_type}_{group}.pkl', 'rb') as input:
    ds = pickle.load(input)

In [9]:
ds

,Array,Chunk
Bytes,732.43 GB,4.72 MB
Shape,"(19, 60934, 158159)","(1, 477, 2472)"
Count,155649 Tasks,155648 Chunks
Type,float32,numpy.ndarray
